# Decentralized Crowd Density (DCD) Maps

First draft of DCD analysis.

## Export structure from OMNeT++

* Each node creates a single file with the map state for each time step
* Global DCD module creates a separate file for the groud 
  truth of the simulation
* todo: write all *node-data* into single file.

### Node map file structure

* only not null values
* first line with `#` contains meta data used for processing in python
* NODE_ID=0a:aa:00:00:00:02 (mac address)
* compound key: [simtime, x, y]
* values:
  * _count_: number of nodes in given cell
  * _measured_t_:   time when count was measured (does not need to be the given node)
  * _received_t_:   time when that measurement was received. If the given node measured the 
                    value itself `received_t` and `simtime` are equal!
  * _source_:       NODE_ID which created the measurement
  * _own_cell_:     If `1` the cell with [x,y] of this row is the current location of the node.
                    Example. node_0a:aa:00:00:00:02 is in cell [66,75] at time 2.0 sec

```
#CELLSIZE=3.000000,DATACOL=-1,IDXCOL=3,NODE_ID=0a:aa:00:00:00:02,SEP=;,XSIZE=281.135000,YSIZE=233.492000
simtime;x;y;count;measured_t;received_t;source;own_cell
2;5;46;1;2;2;0a:aa:00:00:00:02;0
2;24;73;1;2;2;0a:aa:00:00:00:02;0
2;25;73;1;2;2;0a:aa:00:00:00:02;0
2;66;75;1;2;2;0a:aa:00:00:00:02;1
4;5;46;1;4;4;0a:aa:00:00:00:02;0
```

### Global map file structure

* same meta data in first line starting with `#`
* values:
  * same as node map file
  * _node_id_:  String list of node_id's contained in the given cell [x,y]. The list is separated by `,` and 
                not by `;` as indicated by `SEP` in the metadata. This column will be normalized later 
                during python processing. 

```
#CELLSIZE=3.000000,DATACOL=-1,IDXCOL=3,SEP=;,NODE_ID=global,XSIZE=281.135000,YSIZE=233.492000
simtime;x;y;count;measured_t;received_t;source;own_cell;node_id
2;4;46;1;2;2;global;0;0a:aa:00:00:00:08
2;5;46;1;2;2;global;0;0a:aa:00:00:00:04
2;23;72;1;2;2;global;0;0a:aa:00:00:00:03
2;24;73;1;2;2;global;0;0a:aa:00:00:00:06
...
4;5;47;2;4;4;global;0;0a:aa:00:00:00:04,0a:aa:00:00:00:08
```

## First analysis

Class structure and placement in roveranalyzer is not fixed yet.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from roveranalyzer.oppanalyzer.dcd import (
    DcdMap2D,
    DcdMetaData,
    build_global_density_map,
    build_local_density_map,
)

from roveranalyzer.uitls.path import PathHelper
from roveranalyzer.vadereanalyzer.plots.scenario import VaderScenarioPlotHelper

# base paths for example

scenario_path = PathHelper.rover_sim(
        "mucFreiNetdLTE2dMulticast/",
        "vadere00_geo_20201012_2/vadere.d/mf_2peds.scenario",
    ).abs_path()


node_paths = [
    "0a:aa:00:00:00:02",
    "0a:aa:00:00:00:03",
    "0a:aa:00:00:00:04",
    "0a:aa:00:00:00:05",
    "0a:aa:00:00:00:06",
    "0a:aa:00:00:00:07",
    "0a:aa:00:00:00:08",
]

global_path = PathHelper.rover_sim(
        "mucFreiNetdLTE2dMulticast/", "vadere00_geo_20201012_2/global.csv",
    ).abs_path()

* create `VadereScenarioPlotHelper` to add obstacles to map plots. 
* Read csv files into simple pandas.DataFrames (set multiIndex)
  * real_coord=True --> translates cell ids such as [3,5] to the correct values. (e.g with cell_size=3.0 --> [9.0, 15.0]
  * full_map=False  --> do not create missing cells. They will be created lated if needed.

In [ ]:
s_plotter =  VaderScenarioPlotHelper(scenario_path)

node_data = []
for node in node_paths:
    path = PathHelper.rover_sim(
            "mucFreiNetdLTE2dMulticast/", f"vadere00_geo_20201012_2/{node}.csv",
        ).abs_path()
    node_data.append(
        build_local_density_map(path, real_coords=True, full_map=False)
    )

global_data = build_global_density_map(global_path, real_coords=True, with_id_list=True, full_map=False )

### build_XXX_density_map 

* creates meta data object 
* creates dataFrame

In [ ]:
# metadata object
print(global_data[0])
print(global_data[0].node_id)

In [ ]:
# pd.DataFrame (Global)
print(global_data[1].head(5))

#pd.DataFrame (Local)
print(node_data[0][0].node_id)
print(node_data[0][1].head(5))

### DcdMap2D

Simple class to combine an manipulate DCD map data:

* replace NODE_IDs with integers for easier indexing and slicing
* _delay_:    `received_t - measured_t`
* _measurement_age_: `simtime - measured_t`
* _update_age_: `simtime - received_t`

In [ ]:
dcd = DcdMap2D.from_separated_frames(global_data, node_data)
dcd.set_scenario_plotter(s_plotter)

#main pd.DataFrame
dcd.raw2d.head(5)

* descriptive stats over all nodes and cells

In [ ]:
dcd.describe_raw()

* descriptive stats only for global map

In [ ]:
dcd.describe_raw(global_only=True)

## Simple Plots

todo: synchronize color coding!


### Node 3(...:04) Plots

* plot1: Map seen from Node 3(...:04) GREEN  at [15.0, 141.0] [same place as pink 7(...:08)]
* plot2: Node placement taken form global state (ground truth)
* plot3: total count of nodes per simtime. 
  * Blue line is ground truth (7 Nodes)
  * Red Node 3(...:04) is the current node.
  * 'Old' measures are counted. This is the reason for the growing number of nodes.


In [ ]:
p1 = dcd.plot2(4.0, 3, "(data from all 'ages')") 

### Node 3(...:04) Plots at 12.0 second
* density shows 'path'
* look at Node 6(...:07) brown 
  * Node 3 has one measurment [84.0, 84.0],
  * and 3 additional measurment form Node 2(...:03) orange.
   

In [ ]:
p2 = dcd.plot2(12.0, 3, "(data from all 'ages')")

In [ ]:
dcd.raw2d.loc[(3, 12.0),:]

### Set threshold for 'age'

* less 'path' building
* lower spread in plot 3 


In [ ]:
dcd_age = dcd.with_age("measurement_age", 2.0)
p3 = dcd_age.plot2(12.0, 3, title="(measurment_age <= 2.0)")

In [ ]:
dcd.foo()

In [ ]:
dcd_age.raw2d.loc[(3, 12.0),:]